In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
from explorer import Explorer

Additionally, Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:337: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to 

In [3]:
e = Explorer("llama3.1", "webshop_llama")
e.adaptor.initialize_env()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████| 1000/1000 [00:00<00:00, 136276.04it/s]


0 skipped

Loaded 13 goals.


/home/xingkun/webshop/web_agent_site/envs/web_agent_text_env.py:212: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = self._parse_html(html).findAll(text=True)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You can set `disable_env_checker=True` to disable this check.
  logger.warn(


In [4]:
print(e.adaptor.get_env_description())

-----
Init new environment:
[ENV] Webshop
[URL ID]: 9
[INSTRUCTION]: i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 90.00 dollars
-----


In [5]:
e.explore()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Start exploring at 2025-11-12_14:07:06
Step 0
Current state url: http://127.0.0.1:3000/<session_id>
Action status: {'has_search_bar': True, 'clickables': ['search']}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'search[x-large red color women faux fur lined winter warm jacket coat]' is taken
Step 1
Current state url: http://127.0.0.1:3000/search_results/<session_id>/x-large+red+color+women+faux+fur+lined+winter+warm+jacket+coat/1
Action status: {'has_search_bar': False, 'clickables': ['back to search', 'next >', 'b09kp78g37', 'b07zxbgdxf', 'b09msz4vs8', 'b09glvmlms', 'b09c8ynwwb', 'b09s3bn15c', 'b000j3hzxs', 'b07dkgjr74', 'b098xt346y', 'b0957xw92m']}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Action 'click[x-large]' is taken
Step 6
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B0<session_id>KP78G37/x-large+red+women+faux+fur+lined+winter+warm+jacket+coat/1/{"size": "x-large"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'click[x-large]' is taken
Step 7
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B0<session_id>KP78G37/x-large+red+women+faux+fur+lined+winter+warm+jacket+coat/1/{"size": "x-large"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'click[x-large]' is taken
Step 8
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B0<session_id>KP78G37/x-large+red+women+faux+fur+lined+winter+warm+jacket+coat/1/{"size": "x-large"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}
Action 'click[buy now]' is taken
Step 9
Current state url: http://127.0.0.1:3000/done/<session_id>/B0<session_id>KP78G37/{'size': 'x-large'}
Action status: {'has_search_bar': False, 'clickables': []}


In [6]:
e.adaptor.get_state()

{'url': "http://127.0.0.1:3000/done/<session_id>/B0<session_id>KP78G37/{'size': 'x-large', 'color': 'red'}",
 'html': 'Thank you for shopping with us! Your code: None (Paste it in your MTurk interface.) Purchased asin B09KP78G37 options {"color": "red", "size": "x-large"} attrs None category None query None product category None Target asin options attrs price upper instuction text category product category query Goal None Reward Your score (min 0.0, max 1.0) 1.0 Reward Details None'}

In [7]:
e.adaptor.step("click[B09KP78G37]")

In [8]:
e.adaptor.get_state()

{'url': "http://127.0.0.1:3000/done/<session_id>/B0<session_id>KP78G37/{'size': 'x-large', 'color': 'red'}",
 'html': 'Thank you for shopping with us! Your code: None (Paste it in your MTurk interface.) Purchased asin B09KP78G37 options {"color": "red", "size": "x-large"} attrs None category None query None product category None Target asin options attrs price upper instuction text category product category query Goal None Reward Your score (min 0.0, max 1.0) 1.0 Reward Details None'}